In [ ]:
# 📚 **Import Libraries**

import os
import warnings
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
import tensorflow_datasets as tfds
from matplotlib.pyplot import figure
from tensorflow.keras.layers.experimental import preprocessing

In [ ]:
# Random seeds
def set_seed(seed=0):
    np.random.seed(seed)
    tf.random.set_seed(seed)
set_seed()

In [ ]:
# Training data
train_data=pd.read_csv('../input/digit-recognizer/train.csv')

# Test data (scale to be in [0,1])
test_data=pd.read_csv('../input/digit-recognizer/test.csv')/255

# Dimensions
print('Training dataframe dimensions:',train_data.shape)
print('Test dataframe dimensions:',test_data.shape)

# First 5 entries of training data
train_data.head()

In [ ]:
# Labels
y=train_data.label

# Scale features to be in [0,1]
X=train_data.drop('label', axis=1)/255

# Delete to save memory
del train_data

In [ ]:
# Reshape (-1 means unspecified)
X = X.values.reshape(-1, 28, 28, 1)

# Save test data index for submission
test_index = test_data.index

# Reshape
test_data = test_data.values.reshape(-1, 28, 28, 1)

# ***Defining the model and adding callback.***

The callback will stop the training when there is no improvement in the
loss for 30 consecutive epochs.

The restore_best_weights=True will take the model back to its best fit. 

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, verbose=1, restore_best_weights=True)

model = tf.keras.models.Sequential([
  #preprocessing.RandomRotation(factor=0.05, fill_mode='constant'),
  #preprocessing.RandomZoom(height_factor=(-0.05,0.05), width_factor=(-0.05,0.05), fill_mode='constant'),
  tf.keras.layers.Conv2D(16, (2,2), padding = 'same', activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.Conv2D(32, (2,2),  padding = 'same',activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(64, (2,2),  padding = 'same',activation='relu'),
  #tf.keras.layers.Conv2D(128, (2,2),  padding = 'same',activation='relu'),
  #tf.keras.layers.Conv2D(256, (2,2),  padding = 'same',activation='relu'),
  #tf.keras.layers.Conv2D(128, (2,2),  padding = 'same',activation='relu'),
  #tf.keras.layers.Conv2D(64, (2,2),  padding = 'same',activation='relu'),
  tf.keras.layers.Conv2D(32, (2,2),  padding = 'same',activation='relu'),
  tf.keras.layers.Conv2D(16, (2,2),  padding = 'same',activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  #tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])

In [ ]:
#optimizer = 'adamax'

#model.compile(optimizer=optimizer,
#              loss='sparse_categorical_crossentropy',
#              metrics=['accuracy'])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
              #metrics=['sparse_categorical_accuracy'])


history = model.fit(X, 
                    y,
                    epochs=300,
                    verbose=1,
                    callbacks=[callback])

#test_loss, test_acc = model.evaluate(test_images, test_labels)

# **Prepare and send the submission to the output directory**

In [ ]:
test_data = pd.read_csv('../input/digit-recognizer/test.csv')
test_data = np.array(test_data, dtype=np.float32)/255
test_data = test_data.reshape(-1,28,28,1)
prediction = model.predict(test_data)
predict = np.array(np.round(prediction), dtype = np.int32)
predict = np.argmax(predict , axis=1).reshape(-1, 1)
out = [{'ImageId': i+1, 'Label': predict[i][0]} for i in range(len(predict))]
pd.DataFrame(out).to_csv('submission.csv', index=False)

In [ ]:
history.history

In [ ]:
df_loss_acc = pd.DataFrame(history.history)
print(df_loss_acc)

# **This is to remove a submission from the output directory**

In [ ]:
model = tf.keras.models.Sequential([
  #preprocessing.RandomRotation(factor=0.05, fill_mode='constant'),
  #preprocessing.RandomZoom(height_factor=(-0.05,0.05), width_factor=(-0.05,0.05), fill_mode='constant'),
  tf.keras.layers.Conv2D(16, (2,2), padding = 'same', activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.Conv2D(32, (2,2),  padding = 'same',activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(64, (2,2),  padding = 'same',activation='relu'),
  #tf.keras.layers.Conv2D(128, (2,2),  padding = 'same',activation='relu'),
  #tf.keras.layers.Conv2D(256, (2,2),  padding = 'same',activation='relu'),
  #tf.keras.layers.Conv2D(128, (2,2),  padding = 'same',activation='relu'),
  #tf.keras.layers.Conv2D(64, (2,2),  padding = 'same',activation='relu'),
  tf.keras.layers.Conv2D(32, (2,2),  padding = 'same',activation='relu'),
  tf.keras.layers.Conv2D(16, (2,2),  padding = 'same',activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  #tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.summary()

In [ ]:
#os.remove("/kaggle/working/submission.csv")